In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


In [48]:
#Folder containing traffic CSVs
traffic_folder = Path("../../data/raw/traffic")

In [49]:
# Load all monthly traffic data files (March 2025 -> Oct 2025)
traffic_files = sorted(f for f in traffic_folder.glob("traffic_2025-*")if "-10" not in f.name)
traffic_df = pd.concat([pd.read_csv(f) for f in traffic_files], ignore_index=True)

print("Combined traffic data shape:", traffic_df.shape)

Combined traffic data shape: (202663, 11)


In [50]:
# Convert timestamps from string to datetime objects
traffic_df['timestamp'] = pd.to_datetime(traffic_df['timestamp'])
traffic_df = traffic_df.sort_values('timestamp')

In [51]:
# Verify the count of unique undirected segments
unique_segments = traffic_df[['origin', 'destination']].apply(sorted, axis=1).apply(tuple).unique()
print("Number of unique road segments (undirected):", len(unique_segments))
print(unique_segments[:10]) 


Number of unique road segments (undirected): 8
[('Rrethi i Kazermes', 'Rrethi i Maxi 24')
 ('Semaforat Rruga B', 'Xhamia Llapit')
 ('Semaforat Rruga B', 'Xhamia e Llapit') ('Rrethi Flamurit', 'Te Qafa')
 ('Rrethi i Flamurit', 'Te Qafa')
 ('Rrethi i madh (Lakrishte)', 'Rrethi te Komuna e vjeter')
 ('Rrethi te Ismeti', 'Xhamia Llapit')
 ('Rrethi i Flamurit', 'Salla e sporteve 1 Tetori')]


In [52]:
# Normalize casing and whitespace
for col in ['origin', 'destination']:
    traffic_df[col] = (traffic_df[col]
                       .astype(str)
                       .str.strip()
                       .str.replace(r'\s+', ' ', regex=True)
                       .str.lower())

In [53]:
normalized_names = {
    "xhamia llapit": "xhamia e llapit",
    "rrethi flamurit": "rrethi i flamurit",
}

# Apply replacements
traffic_df[['origin', 'destination']] = traffic_df[['origin', 'destination']].replace(normalized_names)

In [54]:
# Create columns for year, month, and date
traffic_df['date'] = traffic_df['timestamp'].dt.date
traffic_df['month'] = traffic_df['timestamp'].dt.to_period('M')

# Count dates per month
days_per_month = traffic_df.groupby('month')['date'].nunique().sort_index()
print(days_per_month)

month
2025-03     6
2025-04    30
2025-05    28
2025-06    20
2025-07    31
2025-08    28
2025-09    30
Freq: M, Name: date, dtype: int64


/tmp/ipykernel_651/1260332266.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  traffic_df['month'] = traffic_df['timestamp'].dt.to_period('M')


In [55]:
# Drop March because it only contains 6 days of data
traffic_df = traffic_df[traffic_df['month'] != pd.Period('2025-03', freq='M')]

Missing Values

In [56]:
# Check how many missing values per column
print(traffic_df.isna().sum())

_id                        0
origin                     0
origin_coords              0
destination                0
destination_coords         0
traffic_speed              0
congestion_level           0
distance                   0
polyline              157002
timestamp                  0
__v                        0
date                       0
month                      0
dtype: int64


Drop Columns that add no modeling value

In [57]:
# _id: random MongoDB object ids - useless for ML
# - __v: legacy versioning field (constant 0 across all rows)
# - polyline: largely missing and unused in current model
# month : only useful for completeness analysis

traffic_df = traffic_df.drop(columns=['__v', 'polyline', '_id', 'month'])

In [58]:
traffic_df.dtypes

origin                             object
origin_coords                      object
destination                        object
destination_coords                 object
traffic_speed                     float64
congestion_level                  float64
distance                            int64
timestamp             datetime64[ns, UTC]
date                               object
dtype: object

In [59]:
# Split coordinate strings into separate float columns
traffic_df[['origin_lat', 'origin_lon']] = traffic_df['origin_coords'].str.split(', ', expand=True).astype(float)
traffic_df[['dest_lat', 'dest_lon']] = traffic_df['destination_coords'].str.split(', ', expand=True).astype(float)


In [60]:
# Drop the original string columns
traffic_df = traffic_df.drop(columns=['origin_coords', 'destination_coords'])

#### Memory Optimization
Downcasting numeric columns to reduce memory and improve computational efficiency

In [61]:
# Downcast numerics
traffic_df['traffic_speed'] = pd.to_numeric(traffic_df['traffic_speed'], downcast='float')
traffic_df['congestion_level'] = pd.to_numeric(traffic_df['congestion_level'], downcast='float')
traffic_df['distance'] = pd.to_numeric(traffic_df['distance'], downcast='integer')

In [62]:
for col in ['origin', 'destination']:
    traffic_df[col] = traffic_df[col].astype('category')

In [63]:
# Sort by timestamp and reset index for chronological analysis
traffic_df = traffic_df.sort_values('timestamp').reset_index(drop=True)

In [64]:
traffic_df.dtypes

origin                         category
destination                    category
traffic_speed                   float32
congestion_level                float32
distance                          int16
timestamp           datetime64[ns, UTC]
date                             object
origin_lat                      float64
origin_lon                      float64
dest_lat                        float64
dest_lon                        float64
dtype: object

In [65]:
traffic_df.shape

(182695, 11)

In [66]:
sampled_df = (traffic_df
              .set_index('timestamp')
              .groupby(['origin', 'destination'], observed=True)  # Fix warning 1
              .resample('20min')  # Fix warning 2
              .agg({
                  'traffic_speed': 'mean',
                  'congestion_level': 'mean',
                  'distance': 'first',  # Static per segment
                  'origin_lat': 'first',
                  'origin_lon': 'first', 
                  'dest_lat': 'first',
                  'dest_lon': 'first'
              })
              .reset_index())

print(f"Sampled data shape: {sampled_df.shape}")
print(f"Unique minutes: {sorted(sampled_df['timestamp'].dt.minute.unique())}")
print(f"Time range: {sampled_df['timestamp'].min()} to {sampled_df['timestamp'].max()}")

Sampled data shape: (158112, 10)
Unique minutes: [np.int32(0), np.int32(20), np.int32(40)]
Time range: 2025-04-01 00:00:00+00:00 to 2025-09-30 23:40:00+00:00


In [67]:
sampled_df['timestamp'] = sampled_df['timestamp'].dt.tz_localize(None)

In [68]:
#Folder containing weather CSVs
weather_folder = Path("../../data/raw/weather")

In [69]:
# Load all monthly weather data files (March 2025 -> Oct 2025)
weather_files = sorted(f for f in weather_folder.glob("weather_2025-*")if "-10" not in f.name)
weather_df = pd.concat([pd.read_csv(f) for f in weather_files], ignore_index=True)

print("Combined weather data shape:", weather_df.shape)

Combined weather data shape: (5136, 3)


In [70]:
weather_df

,time,temperature,precipitation
0,2025-03-01T00:00,5.2,0.0
1,2025-03-01T01:00,5.2,0.0
2,2025-03-01T02:00,5.1,0.0
3,2025-03-01T03:00,5.1,0.0
4,2025-03-01T04:00,5.1,0.0
...,...,...,...
5131,2025-09-30T19:00,10.4,0.0
5132,2025-09-30T20:00,9.1,0.0
5133,2025-09-30T21:00,8.8,0.0
5134,2025-09-30T22:00,8.0,0.0


In [71]:
#Rename time → timestamp and convert to datetime
weather_df = weather_df.rename(columns={'time': 'timestamp'})
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])


In [72]:
weather_df = weather_df[weather_df['timestamp'].dt.month != 3]

In [73]:
#Set timestamp as index for resampling
weather_df = weather_df.set_index('timestamp').sort_index()


In [74]:
# Resample to 20-minute intervals + forward fill
weather_resampled = weather_df.resample('20min').ffill()

In [75]:
weather_resampled.shape

(13174, 2)

In [76]:
traffic_weather_df = sampled_df.merge(weather_resampled, on='timestamp', how='inner')

In [77]:
#Overview of the merged datasets
traffic_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158088 entries, 0 to 158087
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   origin            158088 non-null  category      
 1   destination       158088 non-null  category      
 2   timestamp         158088 non-null  datetime64[ns]
 3   traffic_speed     139730 non-null  float32       
 4   congestion_level  139730 non-null  float32       
 5   distance          139730 non-null  float64       
 6   origin_lat        139730 non-null  float64       
 7   origin_lon        139730 non-null  float64       
 8   dest_lat          139730 non-null  float64       
 9   dest_lon          139730 non-null  float64       
 10  temperature       158088 non-null  float64       
 11  precipitation     158088 non-null  float64       
dtypes: category(2), datetime64[ns](1), float32(2), float64(7)
memory usage: 11.2 MB


### Handle Missing Data 

In [78]:
# Handle missing data
# Sort for consistency
df = traffic_weather_df.sort_values(['origin','destination','timestamp']).copy()


In [79]:
# Interpolate short gaps in dynamic features
dynamic_cols = ['traffic_speed', 'congestion_level']
df[dynamic_cols] = (
    df.groupby(['origin','destination'], observed=True)[dynamic_cols]
      .apply(lambda g: g.interpolate(limit=3).ffill().bfill())
      .reset_index(level=[0,1], drop=True)
)

In [80]:
# Fill static per-edge fields
static_cols = ['distance','origin_lat','origin_lon','dest_lat','dest_lon']
df[static_cols] = (
    df.groupby(['origin','destination'], observed=True)[static_cols]
      .transform(lambda g: g.ffill().bfill())
)

In [81]:
# Confirm no remaining missing values
print(df.isna().sum())

origin              0
destination         0
timestamp           0
traffic_speed       0
congestion_level    0
distance            0
origin_lat          0
origin_lon          0
dest_lat            0
dest_lon            0
temperature         0
precipitation       0
dtype: int64


In [82]:
df[['origin','destination']].drop_duplicates().shape


(12, 2)

### Adding Temporal Features

In [83]:
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df = df.sort_values('timestamp').set_index('timestamp')

# Convert to local timezone
df_local = df.tz_convert("Europe/Belgrade")

In [84]:
# Extract base temporal columns
df['hour'] = df_local.index.hour
df['dayofweek'] = df_local.index.dayofweek  # Monday=0
df['month'] = df_local.index.month
df['is_weekend'] = (df_local.index.dayofweek >= 5).astype('int8')

In [85]:
# Apply cyclical encodings

# Hour of day
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Day of week
df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

# Month (optional — minor seasonal trend)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

In [86]:
# Define official holidays 
holiday_dates = pd.to_datetime([
    "2025-04-09",  # Constitution Day
    "2025-04-21",  # Easter
    "2025-05-01",  # Labour Day
    "2025-05-09",  # Europe Day
    "2025-06-06",  # Eid 
])

# Define Sunny Hill Festival (August 1–3, 2025)
festival_dates = pd.date_range("2025-08-01", "2025-08-03", freq="D")

In [87]:
# Localize both to match the local timezone of the dataframe
holiday_dates_local = pd.DatetimeIndex(holiday_dates).tz_localize("Europe/Belgrade")
festival_dates_local = pd.DatetimeIndex(festival_dates).tz_localize("Europe/Belgrade")

In [88]:
# Create binary columns (1 if date matches)
df["is_holiday"] = df_local.index.normalize().isin(holiday_dates_local).astype("int8")
df["is_event"]   = df_local.index.normalize().isin(festival_dates_local).astype("int8")

In [89]:
# Sunny Hill Festival runs Aug 1–3 2025
festival_dates = pd.date_range("2025-08-01", "2025-08-03", freq="D")

# Whole-day indicator (captures afternoon buildup + late-night)
df["is_event"] = df_local.index.normalize().isin(festival_dates).astype("int8")

In [90]:
# Quick summary check
print("Holiday rows:", int(df["is_holiday"].sum()))
print("Festival rows:", int(df["is_event"].sum()))

Holiday rows: 4320
Festival rows: 0


In [91]:
df

,origin,destination,traffic_speed,congestion_level,distance,origin_lat,origin_lon,dest_lat,dest_lon,temperature,...,month,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,month_sin,month_cos,is_holiday,is_event
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-01 00:00:00+00:00,rrethi i flamurit,salla e sporteve 1 tetori,31.477501,-4.927500,1376.0,42.646832,21.156087,42.653815,21.167529,6.5,...,4,0,0.500000,0.866025,0.781831,0.623490,0.866025,-0.5,0,0
2025-04-01 00:00:00+00:00,rrethi i madh (lakrishte),rrethi te komuna e vjeter,33.197502,-10.655000,2420.0,42.653122,21.146059,42.666980,21.163980,6.5,...,4,0,0.500000,0.866025,0.781831,0.623490,0.866025,-0.5,0,0
2025-04-01 00:00:00+00:00,rrethi i maxi 24,rrethi i kazermes,27.445000,8.515000,3235.0,42.652650,21.175016,42.650434,21.138832,6.5,...,4,0,0.500000,0.866025,0.781831,0.623490,0.866025,-0.5,0,0
2025-04-01 00:00:00+00:00,rrethi te ismeti,xhamia e llapit,29.514999,1.610000,3154.0,42.658080,21.137243,42.671410,21.161750,6.5,...,4,0,0.500000,0.866025,0.781831,0.623490,0.866025,-0.5,0,0
2025-04-01 00:00:00+00:00,rrethi i flamurit,te qafa,24.392500,18.689999,2621.0,42.648019,21.157728,42.665520,21.160670,6.5,...,4,0,0.500000,0.866025,0.781831,0.623490,0.866025,-0.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30 23:00:00+00:00,rrethi i kazermes,rrethi i maxi 24,28.059999,6.480000,3242.0,42.650316,21.138918,42.652456,21.175115,7.1,...,10,0,0.258819,0.965926,0.974928,-0.222521,-0.866025,0.5,0,0
2025-09-30 23:00:00+00:00,rrethi i flamurit,te qafa,21.940001,26.860001,2621.0,42.648019,21.157728,42.665520,21.160670,7.1,...,10,0,0.258819,0.965926,0.974928,-0.222521,-0.866025,0.5,0,0
2025-09-30 23:00:00+00:00,rrethi i flamurit,salla e sporteve 1 tetori,29.660000,1.130000,1376.0,42.646832,21.156087,42.653815,21.167529,7.1,...,10,0,0.258819,0.965926,0.974928,-0.222521,-0.866025,0.5,0,0


In [92]:
# Save the cleaned dataset
output_path = "../../data/processed/traffic_weather_clean.csv"
df.to_csv(output_path, index=True)  # index=True to keep timestamp as a column
print(f"Cleaned dataset saved to: {output_path}")


Cleaned dataset saved to: ../../data/processed/traffic_weather_clean.csv


In [93]:
# Save the cleaned dataset as Parquet (much better than CSV!)
output_path = "../../data/processed/traffic_weather_clean.parquet"
df.to_parquet(output_path, index=True, engine='pyarrow')
print(f"Cleaned dataset saved to: {output_path}")

Cleaned dataset saved to: ../../data/processed/traffic_weather_clean.parquet
